In [ ]:
# !pip install sentence_transformers
# !pip install keras_tuner
# !pip install python-louvain

# from google.colab import drive
# drive.mount('/content/drive')


import csv
import json
import re
import ast
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics.cluster import normalized_mutual_info_score
from sentence_transformers import SentenceTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NeighborhoodComponentsAnalysis
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import NearestCentroid
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix,accuracy_score,ConfusionMatrixDisplay,precision_score,recall_score,f1_score,pairwise_distances
from sklearn.preprocessing import normalize
from imblearn.under_sampling import AllKNN
from imblearn.under_sampling import RandomUnderSampler
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt
import plotly.graph_objects as go
import plotly.express as px
import networkx as nx
import networkx.algorithms.community as nx_comm
from community import community_louvain
from ast import literal_eval
np.set_printoptions(linewidth=25000)

In [ ]:
rawData = []

with open("./booksummaries/booksummaries.txt", 'r') as originalData:
    data = csv.reader(originalData, dialect='excel-tab')
    for row in data:
        rawData.append(row)

#print(rawData[245])
#print(len(rawData))

index = range(len(rawData))

book_data = [[idx,r[0],r[2],r[3],r[4],r[5],r[6]] for idx, r in zip(index, rawData)]

#book_data[0]

df = pd.DataFrame(book_data,columns =['Index','ID','BookTitle','Author','PubDate','Genres_','Summary_'])

#df.info()
df.head(10)

df = df.drop(df[df['Genres_'] == ''].index)
df = df.drop(df[df['Summary_'] == ''].index)

genres_data = []
for g in df['Genres_']:
    genres_data.append(list(json.loads(g).values()))

df['Genres'] = genres_data

genres_list = [i for l in df['Genres'] for i in l]
genres_size = [len(l) for l in df['Genres']]
genre_count = pd.value_counts(genres_list)

df['Genres_Size'] = genres_size

summary_size = [len(l.split(' ')) for l in df['Summary_']]
summary_char = [re.search(r'[^\w\d\s:.,!?]', l) for l in df['Summary_']]

summary_text = []
for s in df['Summary_']:
    summary_text.append(re.sub(r'[^\w\d\s:.,!?]',' ',s))

df['Summary'] = summary_text
df['Summary_Size'] = summary_size

print("Gêneros distintos: ",len(genre_count),"\nContagem de palavras dos sumários, máx ", \
                             max(summary_size)," e mín ",min(summary_size))

selected_genres = ['Science Fiction','Novel','Fantasy',"Children's literature","Mystery"]

In [ ]:
plt.hist(genres_size,rwidth=0.85)
plt.xlabel('Gêneros')
plt.ylabel('Frequência')
plt.title('Distribuição da quantidade de gêneros')
plt.show()


plt.hist(summary_size,np.linspace(0,1500,50),rwidth=0.85)
plt.xlabel('Sumário')
plt.ylabel('Frequência')
plt.title('Distribuição do tamanho em palavras dos sumários')
plt.show()

In [ ]:
df_select = df.drop(df[df['Summary_Size'] > 400].index)
df_select = df_select.drop(df_select[df_select['Summary_Size'] < 50].index)
df_select = df_select.drop(df_select[df_select['Genres_Size'] > 5].index)
df_select = df_select.drop(columns=['Index', 'Genres_', 'Summary_'])
df_select = df_select.reset_index()

df_genre = pd.DataFrame(df_select['Genres'].tolist(),columns=['G1', 'G2', 'G3', 'G4', 'G5'])

df_select = pd.concat([df_select, df_genre], axis = 1)
df_select.loc[~df_select['G1'].isin(selected_genres),'G1'] = 'None'
df_select.loc[~df_select['G2'].isin(selected_genres),'G2'] = 'None'
df_select.loc[~df_select['G3'].isin(selected_genres),'G3'] = 'None'
df_select.loc[~df_select['G4'].isin(selected_genres),'G4'] = 'None'
df_select.loc[~df_select['G5'].isin(selected_genres),'G5'] = 'None'

df_ready = df_select[df_select[['G1','G2','G3','G4','G5']].isin(selected_genres).any(axis = 1)].copy()

def arruma(x):
    if x['G4'] == 'None':
        x['G4'] = x['G5']
        x['G5'] = 'None'
    if x['G3'] == 'None':
        x['G3'] = x['G4']
        x['G4'] = x['G5']
        x['G5'] = 'None'
    if x['G2'] == 'None':
        x['G2'] = x['G3']
        x['G3'] = x['G4']
        x['G4'] = x['G5']
        x['G5'] = 'None'
    if x['G1'] == 'None':
        x['G1'] = x['G2']
        x['G2'] = x['G3']
        x['G3'] = x['G4']
        x['G4'] = x['G5']
        x['G5'] = 'None'
    return x

df_ready = df_ready.apply(lambda x: arruma(x), axis=1)
df_ready['Genres_Class_Size'] = df_ready[df_ready[['G1','G2','G3','G4','G5']].isin(selected_genres)].count(axis=1)


def numera(genres):
    resp = []
    for x in genres:
        if x in selected_genres:
            resp.append(selected_genres.index(x))
    return resp

df_ready['Genre_Class'] = df_ready[['G1','G2','G3','G4','G5']].apply(lambda x: numera(x), axis=1)



plt.hist(df_ready['Genres_Class_Size'],[0.5,1.5,2.5,3.5,4.5,5.5],rwidth=0.85)
plt.xlabel('Gêneros')
plt.ylabel('Frequência')
plt.title('Distribuição da quantidade de gêneros')
plt.show()

df_ready.info()

In [ ]:
df_ready_1g = df_ready[df_ready['Genres_Class_Size'] == 1].copy()
df_ready_ag = df_ready[df_ready['Genres_Class_Size'] > 1].copy()

df_ready_1g.info()
df_ready_ag.info()

In [ ]:
plt.hist(df_ready['Genres_Size'],[0.5,1.5,2.5,3.5,4.5,5.5],rwidth=0.85)
plt.xlabel('Gêneros')
plt.ylabel('Frequência')
plt.title('Distribuição da quantidade de gêneros')
plt.show()


plt.hist(df_ready['Summary_Size'],np.linspace(50,450,20),rwidth=0.85)
plt.xlabel('Sumário')
plt.ylabel('Frequência')
plt.title('Distribuição do tamanho em palavras dos sumários')
plt.show()

In [ ]:
df_train, df_test = train_test_split(df_ready_1g, test_size=0.3, random_state=99)

df_train = df_train.copy()
df_test = df_test.copy()

df_train.to_csv('/content/drive/MyDrive/TCC/df_train.txt')
df_test.to_csv('/content/drive/MyDrive/TCC/df_test.txt')

df_train.head()

In [ ]:
model = SentenceTransformer('/content/drive/MyDrive/TCC/finetune-test')
model.max_seq_length = 512
print("Max Sequence Length:", model.max_seq_length)

df_train = pd.read_csv('/content/drive/MyDrive/TCC/df_train.txt')
df_test = pd.read_csv('/content/drive/MyDrive/TCC/df_test.txt')

df_train = df_train.drop(columns=['Embedding'])
df_test = df_test.drop(columns=['Embedding'])

def embedding(x):
    return model.encode(x)
    
df_train['Embedding'] = df_train['Summary'].apply(lambda x: embedding(x))

df_train.to_csv('/content/drive/MyDrive/TCC/df_train_new.txt')


df_test['Embedding'] = df_test['Summary'].apply(lambda x: embedding(x))

df_test.to_csv('/content/drive/MyDrive/TCC/df_test_new.txt')